## Importing all series into elastic search index

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from elasticsearch_dsl import Mapping, String, Integer
from time import sleep
import pymongo

In [ ]:
es = Elasticsearch()
ies = IndicesClient(es)
db = pymongo.MongoClient().scraper_meta
db.series.count()

In [ ]:
if ies.exists('series'):
    ies.delete('series')
ies.create('series')
sleep(1)

In [ ]:
ies.close('series')
m = Mapping('series')

m.field('accession', String(index='not_analyzed'))
m.field('id', Integer())
m.field('organism', String(index='not_analyzed'))
m.field('platforms', String(index='not_analyzed'))
m.field('samples', String(index='not_analyzed'))

m.field('title', String())
m.field('summary', String())
m.field('overall_design', String())

m.save('series', using=es)

# ies.put_settings(index='series', body={
#     "analysis":{
#       "analyzer":{
#         "default":{
#           "type":"custom",
#           "tokenizer":"standard",
#           "filter":[ "standard", "lowercase", "stop", "kstem" ]
#         }
#       }
#     }
# })
# sleep(1)
ies.open('series')

In [ ]:
def fields(fields_list):
    return dict((f, 1) for f in fields_list)

In [ ]:
series = list(db.series.find({}, fields(['accession', 'organism', 'platforms', 'samples', 'title', 'summary', 'overall_design'])))

In [ ]:
actions = [dict(
            _index='series',
            _type='series',
            _source=dict([('id', int(s['accession'][3:]))] + list((k, v) for (k, v) in s.items() if k != '_id'))
            ) for s in series]


from elasticsearch.helpers import bulk

bulk(es, actions)